In [8]:
from xgboost import XGBClassifier
import warnings
from tabpfn_new.scripts.transformer_prediction_interface import TabPFNClassifier
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from data_prep_utils import *
from evaluate import *
from load_models import *
import matplotlib.pyplot as plt
import torch
import openml
import time

In [9]:
path = "datasets/data_all.csv"
data, labels = get_microbiome(path)
data = top_non_zero(data)
data, labels = unison_shuffled_copies(data, labels)

In [11]:
for sampling in [None, undersample]:
    cv = 3
    strat_split = True
    n_optim = 1000
    ft_epochs = 10
    max_samples = 1000
    metrics = metrics = ["accuracy", "precision", "recall", "roc_auc"]
    models = [
        XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic'),
        XGBoostOptim(n_optim=n_optim),
        LogisticRegression(max_iter=500), 
        TabPFNClassifier(device='cpu', N_ensemble_configurations=5, no_preprocess_mode=True),
        TabForestPFNClassifier("saved_models/tabforest/mix600k/tabforestpfn.pt", "saved_models/tabforest/mix600k/config_run.yaml", max_epochs=ft_epochs)
    ]
    results = pd.DataFrame(np.zeros((len(models), len(metrics)+1)), 
                           index=[m.__class__.__name__ for m in models],
                          columns=metrics+["runtime"])
    
    for ii, model in enumerate(models):
        results.iloc[ii,:] = cross_validate_sample(model, data, labels, metrics, strat_split, cv, sampling, max_samples)
    results_sorted = results.sort_values("roc_auc")
    print(results_sorted)

C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024-10-04 23:10:20.043 |

                        accuracy  precision    recall   roc_auc     runtime
TabPFNClassifier        0.939880   0.000000  0.000000  0.500000    3.995666
XGBoostOptim            0.942552   0.738095  0.088889  0.543023  139.322846
XGBClassifier           0.933200   0.291667  0.155556  0.569249    0.033000
LogisticRegression      0.911156   0.235349  0.188889  0.573122    0.071000
TabForestPFNClassifier  0.940548   0.519814  0.211111  0.599159   24.793001


2024-10-04 23:15:38.821 | INFO     | tabularbench.core.trainer_finetune:train:83 - Epoch 000 | Train loss: -.---- | Train score: -.---- | Val loss: 0.5743 | Val score: 0.7500
2024-10-04 23:15:39.363 | INFO     | tabularbench.core.trainer_finetune:train:94 - Epoch 001 | Train loss: 0.5705 | Train score: 0.6500 | Val loss: 0.6255 | Val score: 0.7083
2024-10-04 23:15:39.856 | INFO     | tabularbench.core.trainer_finetune:train:94 - Epoch 002 | Train loss: 0.4770 | Train score: 0.8000 | Val loss: 0.6387 | Val score: 0.6667
2024-10-04 23:15:40.303 | INFO     | tabularbench.core.trainer_finetune:train:94 - Epoch 003 | Train loss: 0.4497 | Train score: 0.7500 | Val loss: 0.6546 | Val score: 0.6667
2024-10-04 23:15:40.751 | INFO     | tabularbench.core.trainer_finetune:train:94 - Epoch 004 | Train loss: 0.5720 | Train score: 0.6000 | Val loss: 0.6592 | Val score: 0.6667
2024-10-04 23:15:41.195 | INFO     | tabularbench.core.trainer_finetune:train:94 - Epoch 005 | Train loss: 0.5386 | Train sco

                        accuracy  precision    recall   roc_auc    runtime
LogisticRegression      0.604542   0.096905  0.666667  0.633618   0.051334
XGBClassifier           0.671343   0.117626  0.688889  0.679555   0.009000
XGBoostOptim            0.762191   0.152657  0.644444  0.707084  80.545859
TabPFNClassifier        0.726787   0.144723  0.722222  0.724651   0.696001
TabForestPFNClassifier  0.738143   0.153100  0.744444  0.741092   5.052999


In [6]:
print(results_sorted)

                        accuracy  precision    recall   roc_auc     runtime
XGBClassifier           0.728288   0.141617  0.682403  0.706837    0.055000
LogisticRegression      0.738850   0.145708  0.673820  0.708447    0.105000
TabPFNClassifier        0.768264   0.175135  0.753934  0.761565   10.005333
TabForestPFNClassifier  0.784237   0.186522  0.749642  0.768063   29.427220
XGBoostOptim            0.797853   0.201961  0.785408  0.792035  174.394001
